In [7]:
import torch
import mrob
import numpy as np
import pypose as pp

In [8]:
xi = np.random.rand(6)*5
x = np.random.rand(3)*10

T = mrob.geometry.SE3(xi)

In [9]:
def finite_difference_grad(xi, function, eps=1e-9, **kwargs):
    T = mrob.geometry.SE3(xi)
    result = []
    for d_xi in np.eye(6)*eps:
        dT = mrob.geometry.SE3(d_xi)
        L1 = function(dT.mul(T), **kwargs)
        L0 = function(T, **kwargs)
        result.append((L1-L0)/eps)
    return np.array(result[3:] + result[:3]).T

### Gradient of $t_k = Tx_k$ wrt to T

In [10]:
def function1(T, x):
    p = T.transform(x)
    return p @ p.T
    # return p

In [11]:
num1 = finite_difference_grad(xi=xi, function=function1, x = x)

In [12]:
p = T.transform(x)
analyt1 = 2 * p @ np.concatenate((np.eye(3), -mrob.hat3(p)), 1)
# analyt1 = np.concatenate((np.eye(3), -mrob.hat3(p)), 1)

In [13]:
num1

array([ 1.59271991e+01,  1.08368567e+01,  5.80079984e+00,  1.42108547e-05,
       -1.42108547e-05, -2.84217094e-05])

In [14]:
analyt1

array([ 1.59271983e+01,  1.08368614e+01,  5.80080128e+00,  1.00445894e-15,
       -3.03376134e-15,  5.48052066e-15])

In [15]:
class PoseTransform(torch.nn.Module):
     def __init__(self, pose : pp.SE3):
         super().__init__()
         self.pose = pp.Parameter(pose)

     def forward(self, x):
         a = self.pose.Act(x)
         return a @ a.T

In [16]:
T_pp = pp.se3(torch.tensor(xi[3:].tolist() + xi[:3].tolist()).unsqueeze(0).double()).Exp()
# self.pose = pp.Parameter(pose)
T_pp = PoseTransform(pose = T_pp)

x_pp = torch.from_numpy(x).unsqueeze(0)

In [30]:
import pypose as pp
import torch
a = pp.SE3(torch.tensor([1,2,3,4,5,6,9]))

In [31]:
a.matrix()

tensor([[-121,  -68,  138,    1],
        [ 148, -103,  -12,    2],
        [ -42,  132,  -81,    3],
        [   0,    0,    0,    1]])

In [35]:
x = pp.randn_SE3(3, sigma=0.1, requires_grad=True, device="cuda")
assert x.is_leaf

In [38]:
loss = (x.Log()**2).sin().sum() # Just test, No physical meaning
loss.backward()
y = x.detach()
loss

x.grad

tensor([[ 0.7432, -0.5907,  0.1349, -0.2256,  0.7351, -0.4696,  0.0000],
        [ 0.9289, -0.0732,  0.6905,  0.0270, -0.0123,  0.0242,  0.0000],
        [-0.3188,  0.1124, -0.3118, -0.0804,  0.0774,  0.0594,  0.0000]],
       device='cuda:0')

In [40]:
x.add_

SE3Type LieTensor:
LieTensor([[ 0.1237, -0.0985,  0.0226, -0.0188,  0.0612, -0.0390,  0.9972],
           [ 0.1549, -0.0122,  0.1151,  0.0022, -0.0010,  0.0020,  1.0000],
           [-0.0531,  0.0187, -0.0520, -0.0067,  0.0064,  0.0049,  0.9999]],
          device='cuda:0', requires_grad=True)

In [19]:
loss = T_pp(x_pp)

In [20]:
loss.backward()

In [22]:
T_pp.pose.grad

tensor([[15.9272, 10.8369,  5.8008,  0.0000,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64)

In [ ]:
T_pp.

In [22]:
auto1 = pp.optim.functional.modjac(T_pp, x_pp)[0].squeeze().numpy()

In [23]:
np.allclose(auto1[:-1], analyt1)

True

In [24]:
auto1

array([25.59251692, 26.71946168,  7.35441911,  0.        ,  0.        ,
        0.        ,  0.        ])

In [20]:
T_pp.pose[0].data

tensor([ 3.9168,  4.7826,  2.1187, -0.1348, -0.1663, -0.0715, -0.9742],
       dtype=torch.float64)

### Gradient of $p_k = h(P,t)$ projection

In [2]:
P = np.array([[1,0,2,0],
              [0,3,4,0],
              [0,0,5,6],
              [0,0,7,0]])

In [3]:
def h(P,t):
    return np.array([P[0,0]/P[3,2] * (t[0]/t[2]) + P[0,2]/P[3,2],
                     P[1,1]/P[3,2] * (t[1]/t[2]) + P[1,2]/P[3,2]])

In [ ]:
def function3(T, x, P):
    return h(P,T.transform(x))

In [ ]:
num3 = finite_difference_grad(xi, function=function3, x=x, P=P)

In [ ]:
t = T.transform(x)
h_ = h(P, t)
analyt3 = np.array([[P[0,0]/P[3,2]/t[2], 0, -P[0,0]/P[3,2]*t[0]/t[2]**2, -P[0,0]/P[3,2]*t[0]*t[1]/t[2]**2, P[0,0]/P[3,2] + P[0,0]/P[3,2]*t[0]**2/t[2]**2, -P[0,0]/P[3,2]*t[1]/t[2]],
          [0, P[1,1]/P[3,2]/t[2], -P[1,1]/P[3,2]*t[1]/t[2]**2, -P[1,1]/P[3,2] - P[1,1]/P[3,2]*t[1]**2/t[2]**2, P[1,1]/P[3,2]*t[0]*t[1]/t[2]**2, P[1,1]/P[3,2]*t[0]/t[2]]])

In [ ]:
np.allclose(num3, analyt3)

True

In [41]:
import torch

In [51]:
torch.linalg.pinv(torch.randn((960*540*3, 7)).cuda())

tensor([[ 1.8174e-07,  1.2421e-07,  6.6726e-07,  ..., -1.0003e-06,
          1.2430e-06, -5.3569e-07],
        [-1.5877e-07,  2.0543e-07,  3.4050e-07,  ...,  1.3577e-06,
         -1.1193e-07,  6.5651e-08],
        [ 6.8597e-07, -1.5576e-07, -2.4416e-07,  ..., -6.0891e-08,
          1.2766e-06,  7.3729e-07],
        ...,
        [ 1.6734e-06,  1.1445e-06,  1.3751e-06,  ...,  1.3786e-07,
         -1.4341e-06, -7.0434e-08],
        [ 9.6750e-08, -1.5462e-07,  1.4729e-07,  ...,  7.2222e-07,
          1.2641e-06,  2.2514e-07],
        [ 2.0104e-08, -5.9228e-07, -9.6559e-07,  ...,  4.2581e-07,
         -2.8512e-07, -3.1146e-07]], device='cuda:0')

: 